In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import json
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import plotly.graph_objects as go

df_co2_per_capita = pd.read_csv('co-emissions-per-capita.csv')


In [2]:
# Filteren van rijen waar de 'Code' kolom ontbrekende waarden heeft
missing_code_data = df_co2_per_capita[df_co2_per_capita['Code'].isnull()]

# Lijst van 'Entity' waarden zonder corresponderende 'Code'
entities_missing_code = missing_code_data['Entity'].unique()

# Output: alleen maar Continenten, geen landen. Dus deze rijen verwijderen uit de dataset.
# Filteren van de dataset om rijen te behouden waar 'Entity' niet in entities_missing_code zit
df_co2_per_capita_cleaned = df_co2_per_capita[~df_co2_per_capita['Entity'].isin(entities_missing_code)]

In [3]:
def check_chronological(df, entity_col='Entity', year_col='Year'):
    """
    Controleert of de rijen van elk land chronologisch zijn gesorteerd op basis van de kolom 'Year'.
    
    Parameters:
    df (pd.DataFrame): De DataFrame die gecontroleerd moet worden.
    entity_col (str): De naam van de kolom die de entiteiten (landen) bevat.
    year_col (str): De naam van de kolom die de jaren bevat.
    
    Returns:
    bool: True als alle rijen chronologisch zijn gesorteerd, anders False.
    dict: Een dictionary met niet-chronologische entiteiten en hun niet-chronologische jaren.
    """
    non_chronological_entities = {}

    for entity in df[entity_col].unique():
        entity_df = df[df[entity_col] == entity]
        if not entity_df[year_col].is_monotonic_increasing:
            non_chronological_entities[entity] = entity_df[year_col].tolist()
    
    if non_chronological_entities:
        return False, non_chronological_entities
    else:
        return True, {}

# Controleer of de DataFrame chronologisch is
is_chronological, non_chronological_entities = check_chronological(df_co2_per_capita_cleaned)

if is_chronological:
    print("Alle rijen zijn chronologisch gesorteerd.")
else:
    print("De volgende entiteiten hebben niet-chronologische jaren:")
    for entity, years in non_chronological_entities.items():
        print(f"{entity}: {years}")

# Output: Alle rijen zijn chronologisch gesorteerd.

Alle rijen zijn chronologisch gesorteerd.


In [4]:
# Sorteer de dataset op de 'Year' kolom
df_sorted = df_co2_per_capita_cleaned.sort_values(by='Year')

# Controleren op dubbele rijen
duplicate_rows = df_sorted[df_sorted.duplicated()]

# Afdrukken van dubbele rijen
if not duplicate_rows.empty:
    print("Dubbele rijen gevonden:")
    print(duplicate_rows)
else:
    print("Geen dubbele rijen gevonden.")


Geen dubbele rijen gevonden.


In [5]:
# De daadwerkelijke waarden voor de tickvals
actual_tickvals = [0, 1, 2, 5, 10, 15, 20]

# Maak de kaart met een slider voor de jaren
fig = px.choropleth(df_co2_per_capita_cleaned, 
                    locations="Code", 
                    color="Annual CO₂ emissions (per capita)", 
                    hover_name="Entity", 
                    animation_frame="Year",
                    color_continuous_scale=px.colors.sequential.Viridis,
                    range_color=(0, 25),
                    labels={'Annual CO₂ emissions (per capita)': 'CO₂ emissies (per capita)'},
                    title='CO₂ emissie per capita ')

# Aanpassingen aan de layout
fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular',
        lataxis=dict(range=[-60, 90]),  # Zuidpool niet weergeven
        uirevision='fixed'  # Behoud de staat van de kaart tussen updates
    ),
    width=850,  # Vergroot de breedte van de kaart
    height=650,  # Vergroot de hoogte van de kaart
    title={
        'text': "CO₂ emissies per capita",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24}  # Grotere titel
    },
    coloraxis_colorbar=dict(
        title="",
        thicknessmode="pixels",
        thickness=30,
        lenmode="pixels",
        len=850,  # Langer maken van de colorbar
        orientation='h',  # Horizontale colorbar
        xanchor='center',
        x=0.5,
        y=-0.25,  # Plaatsing tussen slider en kaart
        ticks="outside",
        tickvals=actual_tickvals,  # gebruik visueel gelijk verdeelde tickvals
        ticktext=["0 t", "1 t", "2 t", "5 t", "10 t", "15 t", "20 t"]  # de daadwerkelijke waarden
    )
)

# Geen data kleur instellen
fig.update_geos(
    lakecolor="lightblue",
    projection_type="equirectangular",
    showcountries=False,
    countrycolor="gray",
    coastlinecolor="white"
)

# Kleur toelichting voor landen zonder data
fig.add_trace(
    px.choropleth(df_sorted, 
                  locations="Code", 
                  color="Annual CO₂ emissions (per capita)", 
                  hover_name="Entity", 
                  animation_frame="Year",
                  color_continuous_scale=custom_color_scale,
                  range_color=(0, 40)).data[0]
)

fig.data[-1].update(
    showscale=True,
    colorscale=[[0, "rgb(169,169,169)"], [1, "rgb(169,169,169)"]],
    colorbar=dict(
        title="No data",
        ticks="",
        lenmode="pixels",
        len=50,
        y=-0.2,
        yanchor='top',
        x=0.5,
        xanchor='center',
        thickness=30,
        orientation='h'
    )
)

# Unieke jaren verkrijgen voor de slider stappen
unique_years = df_co2_per_capita_cleaned['Year'].unique()
unique_years.sort()  # Zorg ervoor dat de jaren gesorteerd zijn

# Aangepaste slider
fig.update_layout(
    sliders=[{
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 10, 'easing': 'linear'},  # Sneller bewegen door jaren
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.07,
        'y': -0.1,  # Plaatsing van de slider lager om overlap te voorkomen
        'steps': [{
            'args': [[str(year)], {'frame': {'duration': 10, 'redraw': True},
                                   'mode': 'immediate',
                                   'transition': {'duration': 10}}],
            'label': str(year),
            'method': 'animate'
        } for year in unique_years]
    }]
)

# Toon de kaart
fig.show()

NameError: name 'custom_color_scale' is not defined